<div style="  background: linear-gradient(145deg, #0f172a, #1e293b);  border: 4px solid transparent;  border-radius: 14px;  padding: 18px 22px;  margin: 12px 0;  font-size: 26px;  font-weight: 600;  color: #f8fafc;  box-shadow: 0 6px 14px rgba(0,0,0,0.25);  background-clip: padding-box;  position: relative;">  <div style="    position: absolute;    inset: 0;    padding: 4px;    border-radius: 14px;    background: linear-gradient(90deg, #06b6d4, #3b82f6, #8b5cf6);    -webkit-mask:       linear-gradient(#fff 0 0) content-box,       linear-gradient(#fff 0 0);    -webkit-mask-composite: xor;    mask-composite: exclude;    pointer-events: none;  "></div>    <b>Category Embeddings & Shared Layers</b>    <br/>  <span style="color:#9ca3af; font-size: 18px; font-weight: 400;">(Advanced Deep Learning with Keras)</span></div>

## Table of Contents

1. [Category Embeddings: Overview](#section-1)
2. [Defining Inputs](#section-2)
3. [The Embedding Layer](#section-3)
4. [Flattening the Output](#section-4)
5. [Putting It All Together: The Embedding Model](#section-5)
6. [Shared Layers: Concept](#section-6)
7. [Implementing Shared Layers](#section-7)
8. [Sharing Multiple Layers as a Model](#section-8)
9. [Merge Layers: Overview](#section-9)
10. [Implementing Merge Layers (Add)](#section-10)
11. [Creating and Compiling Multi-Input Models](#section-11)
12. [Fitting with Multiple Inputs](#section-12)
13. [Predicting with Multiple Inputs](#section-13)
14. [Evaluating with Multiple Inputs](#section-14)
15. [Conclusion](#section-15)

***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 1. Category Embeddings: Overview</span><br>

Category embeddings are a powerful technique in deep learning for handling categorical data (like team IDs, user IDs, or word indices). Instead of using one-hot encoding, which can result in massive sparse matrices, embeddings map integers to dense vectors of floating-point numbers.

### Key Characteristics
*   **Input**: Integers (representing categories).
*   **Output**: Floats (dense vectors).
*   **Dimensionality**: Embeddings increase dimensionality. The output layer often needs to be flattened back to 2D for subsequent dense layers.

### Architecture Flow
The typical flow for an embedding network is:
`Input Layer (Integer)` $\rightarrow$ `Embedding Layer (Lookup Table)` $\rightarrow$ `Output Layer (Float)`

<div style="background: #e0f2fe; border-left: 16px solid #0284c7; padding: 14px 18px; border-radius: 8px; font-size: 18px; color: #075985;"> 💡 <b>Tip:</b> Think of an embedding layer as a learnable lookup table where the network learns the best vector representation for each category during training. </div>

***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 2. Defining Inputs</span><br>

The first step in building a Keras model using the Functional API is defining the input tensor. For embeddings, the input is typically a single integer per sample.

### Code Implementation



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input

# Define the input tensor
# shape=(1,) indicates a single integer input per sample
input_tensor = Input(shape=(1,))

print(input_tensor)



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 3. The Embedding Layer</span><br>

The `Embedding` layer is the core component. It requires specifying the size of the vocabulary (number of unique categories) and the dimensionality of the embedding vector.

### Parameters
*   `input_dim`: How many unique categories exist (e.g., number of teams).
*   `input_length`: Length of the input sequence (1 in this case).
*   `output_dim`: Size of the vector representing each category.

### Code Implementation



In [ ]:
from tensorflow.keras.layers import Embedding

# Example: Total number of unique teams in the dataset
n_teams = 10887

# Create the embedding layer
embed_layer = Embedding(input_dim=n_teams,
                        input_length=1,
                        output_dim=1,
                        name='Team-Strength-Lookup')

# Apply the embedding layer to the input tensor
embed_tensor = embed_layer(input_tensor)

print(embed_tensor)



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 4. Flattening the Output</span><br>

Embedding layers add a dimension. If you input a shape of `(Batch_Size, 1)`, the embedding layer outputs `(Batch_Size, 1, Output_Dim)`. To pass this to a standard Dense layer or output layer, we often need to flatten it back to 2D.

### Code Implementation



In [ ]:
from tensorflow.keras.layers import Flatten

# Flatten the embedding output
# This converts shape (Batch, 1, 1) -> (Batch, 1)
flatten_tensor = Flatten()(embed_tensor)

print(flatten_tensor)



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 5. Putting It All Together: The Embedding Model</span><br>

We can now combine the Input, Embedding, and Flatten layers into a single Keras Model. This model takes an integer as input and outputs a float representing the "strength" or "value" of that category.

### Code Implementation



In [ ]:
from tensorflow.keras.models import Model

# 1. Input Layer
input_tensor = Input(shape=(1,))

# 2. Define constants
n_teams = 10887

# 3. Embedding Layer
embed_layer = Embedding(input_dim=n_teams,
                        input_length=1,
                        output_dim=1,
                        name='Team-Strength-Lookup')

# 4. Connect layers
embed_tensor = embed_layer(input_tensor)
flatten_tensor = Flatten()(embed_tensor)

# 5. Create the Model
model = Model(input_tensor, flatten_tensor)

# Display model summary
model.summary()



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 6. Shared Layers: Concept</span><br>

Shared layers allow you to use the **same** layer instance (with the same weights) on multiple different inputs. This is useful when you have two inputs that are of the same type and should be processed in the exact same way (e.g., comparing two teams).

### Key Features
*   **Requirement**: Requires the Keras Functional API.
*   **Flexibility**: Extremely flexible architecture design.
*   **Mechanism**: The weights are shared because the *same object* is called on different tensors.

***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 7. Implementing Shared Layers</span><br>

To share a layer, you instantiate it once, and then call that single instance on multiple input tensors.

### Code Implementation



In [ ]:
from tensorflow.keras.layers import Dense

# Define two separate inputs
input_tensor_1 = Input((1,))
input_tensor_2 = Input((1,))

# Create a single shared layer instance
# We do NOT create two Dense layers, just one.
shared_layer = Dense(1)

# Apply the SAME layer to both inputs
output_tensor_1 = shared_layer(input_tensor_1)
output_tensor_2 = shared_layer(input_tensor_2)

print("Output 1:", output_tensor_1)
print("Output 2:", output_tensor_2)



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 8. Sharing Multiple Layers as a Model</span><br>

You can share entire models, not just single layers. Since a Keras `Model` acts like a layer, you can instantiate a model once and call it on multiple inputs. This allows complex preprocessing pipelines (like the embedding pipeline created in Section 5) to be shared.

### Code Implementation



In [ ]:
# --- Step 1: Create the reusable model (from Section 5) ---
input_tensor = Input(shape=(1,))
n_teams = 10887
embed_layer = Embedding(input_dim=n_teams,
                        input_length=1,
                        output_dim=1,
                        name='Team-Strength-Lookup')
embed_tensor = embed_layer(input_tensor)
flatten_tensor = Flatten()(embed_tensor)

# This model encapsulates the embedding and flattening logic
model = Model(input_tensor, flatten_tensor)

# --- Step 2: Use the model as a shared layer ---

# Define two new inputs (e.g., Team A and Team B)
input_tensor_1 = Input((1,))
input_tensor_2 = Input((1,))

# Pass both inputs through the SAME model
output_tensor_1 = model(input_tensor_1)
output_tensor_2 = model(input_tensor_2)

print("Shared Model Output 1:", output_tensor_1)
print("Shared Model Output 2:", output_tensor_2)



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 9. Merge Layers: Overview</span><br>

When you have multiple inputs (like two teams competing), you eventually need to combine them to make a prediction. Keras provides **Merge Layers** for this purpose.

### Common Merge Operations
*   **Add**: Sums inputs element-wise.
*   **Subtract**: Subtracts inputs element-wise.
*   **Multiply**: Multiplies inputs element-wise.
*   **Concatenate**: Joins tensors together along a specific axis.

***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 10. Implementing Merge Layers (Add)</span><br>

The `Add` layer takes a **list** of tensors as input and returns a single tensor containing their sum.

### Code Implementation



In [ ]:
from tensorflow.keras.layers import Add

# Define inputs
in_tensor_1 = Input((1,))
in_tensor_2 = Input((1,))

# Merge them using Add
# Note: The input to Add() is a LIST of tensors
out_tensor = Add()([in_tensor_1, in_tensor_2])

print("Output Tensor (2 inputs):", out_tensor)

# Example with 3 inputs
in_tensor_3 = Input((1,))
out_tensor_3 = Add()([in_tensor_1, in_tensor_2, in_tensor_3])

print("Output Tensor (3 inputs):", out_tensor_3)



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 11. Creating and Compiling Multi-Input Models</span><br>

To create a model with multiple inputs, you pass a **list** of input tensors to the `Model` constructor.

### Code Implementation



In [ ]:
# Define inputs
in_tensor_1 = Input((1,))
in_tensor_2 = Input((1,))

# Merge logic
out_tensor = Add()([in_tensor_1, in_tensor_2])

# Create the model
# inputs argument takes a list: [input_1, input_2]
model = Model(inputs=[in_tensor_1, in_tensor_2], outputs=out_tensor)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

model.summary()



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 12. Fitting with Multiple Inputs</span><br>

When fitting a model with multiple inputs, the `x` argument in `model.fit()` must be a list of numpy arrays (or tensors) that corresponds exactly to the list of input layers defined in the model.

### Code Implementation



In [ ]:
import numpy as np

# Generate dummy data
# 10 samples, 1 feature each
data_1 = np.random.randint(1, 100, size=(10, 1))
data_2 = np.random.randint(1, 100, size=(10, 1))

# Target: Simple sum for demonstration
target = data_1 + data_2

# Fit the model
# Pass inputs as a list: [data_1, data_2]
model.fit([data_1, data_2], target, epochs=1, verbose=1)



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 13. Predicting with Multiple Inputs</span><br>

Prediction follows the same pattern as fitting. You pass a list of numpy arrays to `model.predict()`.

### Code Implementation



In [ ]:
# Example 1: 1 + 2 = 3
# Note: Inputs must be 2D arrays (Batch Size, Features)
pred_1 = model.predict([np.array([[1]]), np.array([[2]])])
print(f"Prediction for 1 + 2: {pred_1[0][0]}")

# Example 2: 42 + 119 = 161
pred_2 = model.predict([np.array([[42]]), np.array([[119]])])
print(f"Prediction for 42 + 119: {pred_2[0][0]}")



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 14. Evaluating with Multiple Inputs</span><br>

Evaluation also requires the list format for inputs.

### Code Implementation



In [ ]:
# Evaluate the model
# Inputs: -1 and -2. Target: -3
# Since the model learned addition, loss should be near 0
loss = model.evaluate([np.array([[-1]]), np.array([[-2]])], 
                      np.array([[-3]]))

print(f"Evaluation Loss: {loss}")



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 15. Conclusion</span><br>

In this notebook, we covered advanced Keras techniques for handling categorical data and complex architectures:

1.  **Category Embeddings**: We learned how to transform integer inputs into dense float vectors using `Embedding` layers, which act as learnable lookup tables.
2.  **Shared Layers**: We utilized the Functional API to apply the same layer (or even an entire model) to multiple inputs, allowing the network to learn shared representations.
3.  **Merge Layers**: We explored how to combine multiple input streams using `Add` (and listed others like `Subtract`, `Multiply`, `Concatenate`).
4.  **Multi-Input Workflow**: We demonstrated the syntax for `fit`, `predict`, and `evaluate` when working with models that require lists of inputs.

**Next Steps**:
*   Apply embeddings to real-world datasets like recommender systems.
*   Experiment with `Concatenate` to combine embeddings with other numerical features.
*   Use shared layers to build "Siamese Networks" for comparing similarity between inputs.
